In [1]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import skimage.io as io
import os, sys
from matplotlib import pyplot as plt

# Add a path to a custom fork of TF-Slim
# Get it from here:
sys.path.append("/home/shou/network/tf-models/research/slim")

# Add path to the cloned library
sys.path.append("/home/shou/network/fcn/tf-image-segmentation")

checkpoints_dir = '/home/shou/network/checkpoints'
log_folder = '/home/shou/network/log_folder_fcn32s'


slim = tf.contrib.slim
vgg_checkpoint_path = os.path.join(checkpoints_dir, 'vgg_16.ckpt')

from tf_image_segmentation.utils.tf_records import read_tfrecord_and_decode_into_image_annotation_pair_tensors
from tf_image_segmentation.models.fcn_32s import FCN_32s, extract_vgg_16_mapping_without_fc8

from tf_image_segmentation.utils.pascal_voc import pascal_segmentation_lut

from tf_image_segmentation.utils.training import get_valid_logits_and_labels

from tf_image_segmentation.utils.augmentation import (distort_randomly_image_color,
                                                      flip_randomly_left_right_image_with_annotation,
                                                      scale_randomly_image_with_annotation_with_fixed_size_output)

In [2]:
image_train_size = [384, 384]
number_of_classes = 2
# tfrecord_filename = '/home/shou/network/island/pascal_augmented_train_pas.tfrecords'
tfrecord_filename = '/home/shou/network/dataset/pascal_augmented_train_island.tfrecords'
pascal_voc_lut = pascal_segmentation_lut(number_of_classes)
class_labels = pascal_voc_lut.keys()
print(class_labels)

[0, 1, 255]


In [3]:
filename_queue = tf.train.string_input_producer(
    [tfrecord_filename], num_epochs=5)

image, annotation = read_tfrecord_and_decode_into_image_annotation_pair_tensors(filename_queue)

# Various data augmentation stages
image, annotation = flip_randomly_left_right_image_with_annotation(image, annotation)

# image = distort_randomly_image_color(image)

resized_image, resized_annotation = scale_randomly_image_with_annotation_with_fixed_size_output(image, annotation, image_train_size)


resized_annotation = tf.squeeze(resized_annotation)

# 参与训练的图像有486幅，验证208幅
image_batch, annotation_batch = tf.train.shuffle_batch( [resized_image, resized_annotation],
                                             batch_size=1,
                                             capacity=3000,
                                             num_threads=2,
                                             min_after_dequeue=1000)

upsampled_logits_batch, vgg_16_variables_mapping = FCN_32s(image_batch_tensor=image_batch,
                                                           number_of_classes=number_of_classes,
                                                           is_training=True)


valid_labels_batch_tensor, valid_logits_batch_tensor = get_valid_logits_and_labels(annotation_batch_tensor=annotation_batch,
                                                                                     logits_batch_tensor=upsampled_logits_batch,
                                                                                    class_labels=class_labels)


cross_entropies = tf.nn.softmax_cross_entropy_with_logits(logits=valid_logits_batch_tensor,
                                                          labels=valid_labels_batch_tensor)

# Normalize the cross entropy -- the number of elements
# is different during each step due to mask out regions
cross_entropy_sum = tf.reduce_mean(cross_entropies)

pred = tf.argmax(upsampled_logits_batch, dimension=3)
 
probabilities = tf.nn.softmax(upsampled_logits_batch)

Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
Use the `axis` argument instead


In [4]:
with tf.variable_scope("adam_vars"):
    train_step = tf.train.AdamOptimizer(learning_rate=0.000001).minimize(cross_entropy_sum)


# Variable's initialization functions
vgg_16_without_fc8_variables_mapping = extract_vgg_16_mapping_without_fc8(vgg_16_variables_mapping)


init_fn = slim.assign_from_checkpoint_fn(model_path=vgg_checkpoint_path,
                                         var_list=vgg_16_without_fc8_variables_mapping)

global_vars_init_op = tf.global_variables_initializer()

tf.summary.scalar('cross_entropy_loss', cross_entropy_sum)

merged_summary_op = tf.summary.merge_all()

summary_string_writer = tf.summary.FileWriter(log_folder)

# Create the log folder if doesn't exist yet
if not os.path.exists(log_folder):
     os.makedirs(log_folder)
    
#The op for initializing the variables.
local_vars_init_op = tf.local_variables_initializer()

combined_op = tf.group(local_vars_init_op, global_vars_init_op)

# We need this to save only model variables and omit
# optimization-related and other variables.
model_variables = slim.get_model_variables()
saver = tf.train.Saver(model_variables)

In [5]:
with tf.Session()  as sess:
    
    sess.run(combined_op)
    init_fn(sess)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # 10 epochs
    for i in xrange(5 * 486):
    
        cross_entropy, summary_string, _ = sess.run([ cross_entropy_sum,
                                                      merged_summary_op,
                                                      train_step ])
        
        print(str(i) + " Current loss: " + str(cross_entropy))
        
        summary_string_writer.add_summary(summary_string, i)
        
        if i % 100 == 0:
            save_path = saver.save(sess, "/home/shou/network/dataset/model_fcn32s.ckpt")
            print("Model saved in file: %s" % save_path)
            
        # if cross_entropy < 0.01:
            # print('the last loss: ' + str(cross_entropy))
            # break

    coord.request_stop()
    coord.join(threads)
    
    save_path = saver.save(sess, "/home/shou/network/dataset/model_fcn32s.ckpt")
    print("Model saved in file: %s" % save_path)
    
summary_string_writer.close()

INFO:tensorflow:Restoring parameters from /home/shou/network/checkpoints/vgg_16.ckpt
0 Current loss: 0.962241
Model saved in file: /home/shou/network/dataset/model_fcn32s.ckpt
1 Current loss: 1.04037
2 Current loss: 0.819615
3 Current loss: 1.04074
4 Current loss: 0.645049
5 Current loss: 0.908365
6 Current loss: 0.802818
7 Current loss: 0.454949
8 Current loss: 0.845721
9 Current loss: 0.842355
10 Current loss: 0.74867
11 Current loss: 0.551187
12 Current loss: 0.717687
13 Current loss: 0.352301
14 Current loss: 0.444706
15 Current loss: 0.603142
16 Current loss: 0.404764
17 Current loss: 0.845797
18 Current loss: 0.414503
19 Current loss: 0.680262
20 Current loss: 0.747489
21 Current loss: 1.24813
22 Current loss: 0.462168
23 Current loss: 0.375206
24 Current loss: 0.814243
25 Current loss: 0.639746
26 Current loss: 0.615388
27 Current loss: 0.522025
28 Current loss: 0.241946
29 Current loss: 0.568578
30 Current loss: 0.440214
31 Current loss: 0.787629
32 Current loss: 1.5283
33 Curr

299 Current loss: 0.530385
300 Current loss: 0.394395
Model saved in file: /home/shou/network/dataset/model_fcn32s.ckpt
301 Current loss: 0.150243
302 Current loss: 0.472794
303 Current loss: 0.344217
304 Current loss: 0.460948
305 Current loss: 0.389735
306 Current loss: 0.380983
307 Current loss: 0.27786
308 Current loss: 0.292346
309 Current loss: 0.270579
310 Current loss: 0.200873
311 Current loss: 0.366163
312 Current loss: 0.402546
313 Current loss: 0.280577
314 Current loss: 0.318702
315 Current loss: 0.218849
316 Current loss: 0.319986
317 Current loss: 0.30831
318 Current loss: 0.513178
319 Current loss: 0.39224
320 Current loss: 0.371212
321 Current loss: 0.206367
322 Current loss: 0.198604
323 Current loss: 0.592414
324 Current loss: 0.326028
325 Current loss: 0.382548
326 Current loss: 0.353293
327 Current loss: 0.301159
328 Current loss: 0.326212
329 Current loss: 0.334365
330 Current loss: 0.473193
331 Current loss: 0.341171
332 Current loss: 0.474151
333 Current loss: 0

596 Current loss: 0.312359
597 Current loss: 0.151314
598 Current loss: 0.16428
599 Current loss: 0.277909
600 Current loss: 0.176244
Model saved in file: /home/shou/network/dataset/model_fcn32s.ckpt
601 Current loss: 0.175539
602 Current loss: 0.103916
603 Current loss: 0.161528
604 Current loss: 0.154715
605 Current loss: 0.118581
606 Current loss: 0.31539
607 Current loss: 0.316464
608 Current loss: 0.304313
609 Current loss: 0.17911
610 Current loss: 0.0758371
611 Current loss: 0.257924
612 Current loss: 0.222256
613 Current loss: 0.100111
614 Current loss: 0.0944436
615 Current loss: 0.310411
616 Current loss: 0.202271
617 Current loss: 0.375138
618 Current loss: 0.240207
619 Current loss: 0.229374
620 Current loss: 0.184851
621 Current loss: 0.317555
622 Current loss: 0.237562
623 Current loss: 0.205002
624 Current loss: 0.178225
625 Current loss: 0.297271
626 Current loss: 0.170482
627 Current loss: 0.328573
628 Current loss: 0.171967
629 Current loss: 0.223178
630 Current loss:

893 Current loss: 0.261998
894 Current loss: 0.0852241
895 Current loss: 0.217967
896 Current loss: 0.170936
897 Current loss: 0.158181
898 Current loss: 0.206248
899 Current loss: 0.0797885
900 Current loss: 0.225056
Model saved in file: /home/shou/network/dataset/model_fcn32s.ckpt
901 Current loss: 0.381899
902 Current loss: 0.216546
903 Current loss: 0.394983
904 Current loss: 0.362622
905 Current loss: 0.234929
906 Current loss: 0.295397
907 Current loss: 0.0988799
908 Current loss: 0.0754561
909 Current loss: 0.25038
910 Current loss: 0.0830887
911 Current loss: 0.0997025
912 Current loss: 0.136364
913 Current loss: 0.219011
914 Current loss: 0.12771
915 Current loss: 0.195058
916 Current loss: 0.172071
917 Current loss: 0.157006
918 Current loss: 0.130112
919 Current loss: 0.219777
920 Current loss: 0.246063
921 Current loss: 0.167105
922 Current loss: 0.143293
923 Current loss: 0.0740081
924 Current loss: 0.284351
925 Current loss: 0.201696
926 Current loss: 0.176155
927 Current

1182 Current loss: 0.150889
1183 Current loss: 0.183995
1184 Current loss: 0.11092
1185 Current loss: 0.203874
1186 Current loss: 0.158574
1187 Current loss: 0.320064
1188 Current loss: 0.118051
1189 Current loss: 0.0448529
1190 Current loss: 0.2125
1191 Current loss: 0.238904
1192 Current loss: 0.262821
1193 Current loss: 0.156704
1194 Current loss: 0.216943
1195 Current loss: 0.229899
1196 Current loss: 0.229452
1197 Current loss: 0.152403
1198 Current loss: 0.0590798
1199 Current loss: 0.259433
1200 Current loss: 0.146995
Model saved in file: /home/shou/network/dataset/model_fcn32s.ckpt
1201 Current loss: 0.284183
1202 Current loss: 0.417869
1203 Current loss: 0.120563
1204 Current loss: 0.166885
1205 Current loss: 0.0239138
1206 Current loss: 0.131835
1207 Current loss: 0.131681
1208 Current loss: 0.291544
1209 Current loss: 0.198745
1210 Current loss: 0.174198
1211 Current loss: 0.0855128
1212 Current loss: 0.146259
1213 Current loss: 0.362961
1214 Current loss: 0.565811
1215 Curr

1468 Current loss: 0.129875
1469 Current loss: 0.241182
1470 Current loss: 0.106819
1471 Current loss: 0.157717
1472 Current loss: 0.170687
1473 Current loss: 0.24785
1474 Current loss: 0.234462
1475 Current loss: 0.185764
1476 Current loss: 0.126358
1477 Current loss: 0.202022
1478 Current loss: 0.209334
1479 Current loss: 0.133
1480 Current loss: 0.232068
1481 Current loss: 0.0791362
1482 Current loss: 0.173277
1483 Current loss: 0.0616081
1484 Current loss: 0.0474811
1485 Current loss: 0.191615
1486 Current loss: 0.223106
1487 Current loss: 0.234079
1488 Current loss: 0.0854098
1489 Current loss: 0.126041
1490 Current loss: 0.128314
1491 Current loss: 0.162005
1492 Current loss: 0.118647
1493 Current loss: 0.158296
1494 Current loss: 0.149768
1495 Current loss: 0.234405
1496 Current loss: 0.199292
1497 Current loss: 0.161563
1498 Current loss: 0.225669
1499 Current loss: 0.0529724
1500 Current loss: 0.082212
Model saved in file: /home/shou/network/dataset/model_fcn32s.ckpt
1501 Curr

1753 Current loss: 0.124595
1754 Current loss: 0.151564
1755 Current loss: 0.125499
1756 Current loss: 0.0617441
1757 Current loss: 0.102297
1758 Current loss: 0.199031
1759 Current loss: 0.126799
1760 Current loss: 0.0294046
1761 Current loss: 0.128658
1762 Current loss: 0.124314
1763 Current loss: 0.361691
1764 Current loss: 0.0844269
1765 Current loss: 0.228747
1766 Current loss: 0.174255
1767 Current loss: 0.152959
1768 Current loss: 0.0821263
1769 Current loss: 0.11405
1770 Current loss: 0.218106
1771 Current loss: 0.20272
1772 Current loss: 0.0964683
1773 Current loss: 0.137633
1774 Current loss: 0.303667
1775 Current loss: 0.108049
1776 Current loss: 0.180324
1777 Current loss: 0.199858
1778 Current loss: 0.174685
1779 Current loss: 0.207551
1780 Current loss: 0.143402
1781 Current loss: 0.199422
1782 Current loss: 0.160179
1783 Current loss: 0.11937
1784 Current loss: 0.0869082
1785 Current loss: 0.35643
1786 Current loss: 0.0604919
1787 Current loss: 0.16748
1788 Current loss:

2038 Current loss: 0.1752
2039 Current loss: 0.191028
2040 Current loss: 0.292079
2041 Current loss: 0.110352
2042 Current loss: 0.153809
2043 Current loss: 0.0901339
2044 Current loss: 0.126975
2045 Current loss: 0.181467
2046 Current loss: 0.0457708
2047 Current loss: 0.283436
2048 Current loss: 0.139985
2049 Current loss: 0.0923589
2050 Current loss: 0.10654
2051 Current loss: 0.269059
2052 Current loss: 0.09207
2053 Current loss: 0.133099
2054 Current loss: 0.102723
2055 Current loss: 0.213153
2056 Current loss: 0.0713294
2057 Current loss: 0.143424
2058 Current loss: 0.19004
2059 Current loss: 0.145543
2060 Current loss: 0.24821
2061 Current loss: 0.0772678
2062 Current loss: 0.132116
2063 Current loss: 0.131393
2064 Current loss: 0.180165
2065 Current loss: 0.131255
2066 Current loss: 0.124359
2067 Current loss: 0.0476192
2068 Current loss: 0.0951587
2069 Current loss: 0.0580695
2070 Current loss: 0.177678
2071 Current loss: 0.218807
2072 Current loss: 0.154293
2073 Current loss:

2323 Current loss: 0.198078
2324 Current loss: 0.136412
2325 Current loss: 0.0835688
2326 Current loss: 0.119551
2327 Current loss: 0.156146
2328 Current loss: 0.191333
2329 Current loss: 0.0791891
2330 Current loss: 0.113015
2331 Current loss: 0.0293854
2332 Current loss: 0.164015
2333 Current loss: 0.171183
2334 Current loss: 0.0531887
2335 Current loss: 0.0845171
2336 Current loss: 0.284454
2337 Current loss: 0.0193769
2338 Current loss: 0.188901
2339 Current loss: 0.0600668
2340 Current loss: 0.13988
2341 Current loss: 0.385531
2342 Current loss: 0.171641
2343 Current loss: 0.0895903
2344 Current loss: 0.118974
2345 Current loss: 0.145949
2346 Current loss: 0.108965
2347 Current loss: 0.210365
2348 Current loss: 0.135912
2349 Current loss: 0.148227
2350 Current loss: 0.177534
2351 Current loss: 0.10156
2352 Current loss: 0.0552614
2353 Current loss: 0.0842753
2354 Current loss: 0.118921
2355 Current loss: 0.104359
2356 Current loss: 0.142017
2357 Current loss: 0.111415
2358 Current